In [12]:
%env  WORKDIR=/tmp/vault

env: WORKDIR=/tmp/vault


In [13]:
%%bash
cat $WORKDIR/VAULT_TOKEN.TXT
cat $WORKDIR/VAULT_ADDR.TXT

hvs.QbtInWOOYqrZJnMrDmcaArKs
https://vault-primary-europe-southwest1-jlhb.josemerchan-f986bf.gcp.sbx.hashicorpdemo.com:8200


Update the values of the enviromental values of the following cells based on the output of the previous cell

In [14]:
%env VAULT_ADDR=https://vault-primary-europe-southwest1-jlhb.josemerchan-f986bf.gcp.sbx.hashicorpdemo.com:8200
%env VAULT_TOKEN=hvs.QbtInWOOYqrZJnMrDmcaArKs

env: VAULT_ADDR=https://vault-primary-europe-southwest1-jlhb.josemerchan-f986bf.gcp.sbx.hashicorpdemo.com:8200
env: VAULT_TOKEN=hvs.QbtInWOOYqrZJnMrDmcaArKs


## Step 1. Install MongoDB

In [9]:
! kubectl apply -f mongodb_deploy.yaml

namespace "mongo" deleted
statefulset.apps "mongo" deleted
service "mongo" deleted


In [10]:
! kubectl apply -f mongodb_deploy.yaml

namespace/mongodb created
deployment.apps/mongodb created
service/mongodb created


# Step 2. Configure Vault

In [ ]:
# Enable Secret Engine
! vault secrets enable -path=mongodb database

Success! Enabled the database secrets engine at: mongodb/


In [ ]:
%%bash
export MONGO_ADDR=$(kubectl get svc -n mongodb mongodb -o json | jq  -r '.status.loadBalancer.ingress[0].ip')

# Create Connection
vault write mongodb/config/mongodb \
    plugin_name=mongodb-database-plugin \
    allowed_roles="*"  connection_url="mongodb://{{username}}:{{password}}@${MONGO_ADDR}:27017/admin?tls=false" \
    username="mdbadmin" \
    password="Passw0rd123!"

Success! Data written to: mongodb/config/mongodb


In [21]:
%%bash
# Create Role
vault write mongodb/roles/mongo-test \
    db_name=mongodb \
    creation_statements='{ "db": "admin", "roles": [{ "role": "readWrite" }, {"role": "read", "db": "foo"}] }' \
    default_ttl="1h" \
    max_ttl="24h"



Success! Data written to: mongodb/roles/mongo-test


In [22]:
# Create Creds
! vault read mongodb/creds/mongo-test

Key                Value
---                -----
lease_id           mongodb/creds/mongo-test/8sCFJhks24SfZk1oBhNSUnY8
lease_duration     1h
lease_renewable    true
password           5gGZvZ-llwiXTieQryxG
username           v-root-mongo-test-p9vrMOg3fGXlyTsr7GdS-1732016396
